In [105]:
import numpy as np

In [106]:
# Wybramo funckje: Sphere i Matyas

def Sphere(X):
    return np.sum(X*X, axis = 1).reshape(-1,1)

def Matyas(X):
   return (0.26 * (Sphere(X)) - (0.48 * X[:, 0] * X[:, 1]).reshape(-1,1)).reshape(-1,1)

Matyas zostal zmieniony, sprawdzic czy jest g, 0,48 zmienione, oblukac w sprawku

In [107]:
def eval(X, function = "sphere",epsilon = 0.001):
    if(function == "sphere"):
        return 1/(Sphere(X)+epsilon).reshape(-1,1)
    elif(function == "matyas"):
        return 1/(Matyas(X)+epsilon).reshape(-1,1)

In [108]:
N = 100

# Tworzenie populacji początkowej i wyliczanie jej przystosowania 
X_sphere = np.random.uniform(low = -5.12, high = 5.12, size = (100,2))
Y_sphere = eval(X_sphere)
X_matyas = np.random.uniform(low = -10, high = 10, size = (100,2))
Y_matyas = eval(X_matyas, "matyas")

In [109]:
# Wybrano skalowanie liniowe i potęgowe
def linear_scaling(X, C = 4, function = "sphere"):
    eval_X = eval(X,function)
    eval_min = np.min(eval_X)
    eval_max = np.max(eval_X)
    F_mean = np.sum(eval_X) / eval_X.shape[0]
    treshold = ( C * F_mean - eval_max ) / ( C - 1.0 )

    if( eval_min > treshold ):
        a = ( ((C-1.0) * F_mean ) / ( eval_max - F_mean) )
        b = F_mean * (eval_max - C * F_mean) / (eval_max - F_mean)
    else:
        a = F_mean / (F_mean - eval_min)
        b = -eval_min * (F_mean) / (F_mean - eval_min)
    
    return a * eval_X + b

def exponential_scaling(X,t, T, function = "sphere", p1 = 0.05, p2 = 0.1, a = 0.1, s_star = 0.1):
    eval_X = eval(X, function)
    s0 = np.max(eval_X) - np.min(eval_X) # według wskazań prowadzącego
    k1 = (s_star/s0) ** p1
    k2 = np.tan(t/(T+1) * np.pi / 2)
    k = k1 * k2 ** (p2*((s0/s_star)**a))
    return np.power(eval_X,k)

In [110]:
Y_sphere_linear = linear_scaling(X_sphere, function = "sphere")
Y_sphere_exponential = exponential_scaling(X_sphere, 1, 100, function = "sphere") # Zakładamy 100 pokoleń

Y_matyas_linear = linear_scaling(X_matyas, function = "matyas")
Y_matyas_exponential = exponential_scaling(X_matyas, 1, 100, function = "matyas") # Zakładamy 100 pokoleń

# 2 bez skalowania
# 2 ze skalowaniem liniowym
# 2 ze skalowaniem potęgowym

In [111]:
def deterministic_choice(X, eval_X):
    N = eval_X.shape[0]
    F_sum = np.sum(eval_X)

    P = (eval_X / F_sum)
    E = P * N
    E_floor = np.floor(E)
    R = E - E_floor
    
    E_floor_sum = int(np.sum(E_floor))
    print(E_floor_sum)
    free_slots = N - E_floor_sum

    X_new_list = []
    c = 0
    for i in range(N):
        for j in range(int(E_floor[i])):
            X_new_list.append(X[i])

    dt = np.concatenate((X,R), axis = 1)
    sorted_dt = dt[dt[:, 2].argsort()[::-1]]

    for i in range(free_slots):
        X_new_list.append(sorted_dt[i, :X.shape[1]])
        
    X_new = np.array(X_new_list)
    return X_new

In [112]:
X_sphere_linear_new = deterministic_choice(X_sphere, Y_sphere_linear)
X_sphere_exponential_new = deterministic_choice(X_sphere, Y_sphere_exponential)
X_sphere_new = deterministic_choice(X_sphere, Y_sphere)

X_matyas_linear_new = deterministic_choice(X_matyas, Y_matyas_linear)
X_matyas_exponential_new = deterministic_choice(X_matyas, Y_matyas_exponential)
X_matyas_new = deterministic_choice(X_matyas, Y_matyas)

20
37
57
20
50
74


/var/folders/xj/dt4pc8px04vc_s_6x99rfymw0000gn/T/ipykernel_34954/1016151733.py:17: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  for j in range(int(E_floor[i])):


In [113]:
def check_subset(X, X_new, name=""):
    X_set = set(map(tuple, X))
    X_new_set = set(map(tuple, X_new))
    if not X_new_set.issubset(X_set):
        raise ValueError(f"Some elements in {name} are not in original {name.split('_')[0]}!")

check_subset(X_sphere, X_sphere_linear_new, "X_sphere_linear_new")
check_subset(X_sphere, X_sphere_exponential_new, "X_sphere_exponential_new")
check_subset(X_sphere, X_sphere_new, "X_sphere_new")

check_subset(X_matyas, X_matyas_linear_new, "X_matyas_linear_new")
check_subset(X_matyas, X_matyas_exponential_new, "X_matyas_exponential_new")
check_subset(X_matyas, X_matyas_new, "X_matyas_new")

In [114]:
print(Sphere(X_sphere).sum())
print(Sphere(X_sphere_new).sum())

1886.0795473279225
595.8660128131843


In [115]:
print(Sphere(X_sphere).sum())
print(Sphere(X_sphere_exponential_new).sum())

1886.0795473279225
1371.8427978694917


In [116]:
print(Sphere(X_sphere).sum())
print(Sphere(X_sphere_linear_new).sum())

1886.0795473279225
1706.0485413951876


In [117]:
print(Matyas(X_matyas).sum())
print(Matyas(X_matyas_new).sum())

1624.141423220708
91.8226089769807


In [118]:
print(Matyas(X_matyas).sum())
print(Matyas(X_matyas_exponential_new).sum())

1624.141423220708
613.4543443776147


In [119]:
print(Sphere(X_matyas).sum())
print(Sphere(X_matyas_linear_new).sum())

7122.883772015767
6483.015958360352
